In [ ]:
import matplotlib.pyplot as plt  # type: ignore
from dotenv import load_dotenv
import sqlalchemy as sq
import seaborn as sns  # type: ignore
import pandas as pd  # type: ignore
import numpy as np
import os, sys

sys.path.append("../")
from Shared.DataService import DataService

In [ ]:
load_dotenv()
PG_DB = os.getenv("POSTGRES_DB")
PG_ADDR = os.getenv("POSTGRES_ADDR")
PG_PORT = os.getenv("POSTGRES_PORT")
PG_USER = os.getenv("POSTGRES_USER")
PG_PW = os.getenv("POSTGRES_PW")

In [ ]:
if (
    PG_DB is None
    or PG_ADDR is None
    or PG_PORT is None
    or PG_USER is None
    or PG_PW is None
):
    raise ValueError("Environment variables not set")

db = DataService(PG_DB, PG_ADDR, int(PG_PORT), PG_USER, PG_PW)
conn = db.connect()

In [ ]:
weatherDataQuery = sq.text(
    """
    SELECT * FROM public.ab_dly_station_data
    UNION
    SELECT * FROM public.mb_dly_station_data
    UNION
    SELECT * FROM public.sk_dly_station_data;
    """
)

dlyDF = pd.read_sql_query(weatherDataQuery, conn)

In [ ]:
stationDataQuery = sq.text(
    """
    SELECT station_id, district FROM public.stations_dly
    WHERE district IS NOT NULL;
    """
)

stationData = pd.read_sql(stationDataQuery, conn)
stationData[["district"]] = stationData[["district"]].astype(int)

In [ ]:
dlyDF = dlyDF.merge(stationData, on="station_id")

In [ ]:
ergotQuery = sq.text(
    """
    SELECT year, district, percnt_true, has_ergot, sum_severity, present_prev1, present_prev2, present_prev3, present_in_neighbor, severity_prev1, 
        severity_prev2, severity_prev3, severity_in_neighbor FROM public.agg_ergot_samples
    GROUP BY year, district, percnt_true, has_ergot, sum_severity, present_prev1, present_prev2, present_prev3, present_in_neighbor, severity_prev1, 
        severity_prev2, severity_prev3, severity_in_neighbor;
    """
)

ergotDF = pd.read_sql_query(ergotQuery, conn)

In [ ]:
final_df = dlyDF.merge(ergotDF, on=["year", "district"])

In [ ]:
final_df.columns

In [ ]:
final_df.drop(columns=["station_id", "date", "month", "day", "year"], inplace=True)

In [ ]:
# Compute the correlation matrix
corr = final_df.corr()

In [ ]:
corr

In [ ]:
sns.set_theme(style="white")

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(
    corr,
    mask=mask,
    cmap=cmap,
    vmax=0.1,
    center=0,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.5},
)

In [ ]:
db.cleanup()